In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Quasi Majoranas in 3D


In [ ]:
import hpc05

client, dview, lview = hpc05.start_remote_and_connect(300, profile='pbs',
                                               folder='~/Work/induced_gap_B_field/',
                                               timeout=300, del_old_ipcluster=True)

# client, dview, lview = hpc05.connect.connect_ipcluster(50, profile='pbs2',
#                                                       folder='~/Work/induced_gap_B_field/',
#                                                       timeout=300)


In [ ]:
# 1. Standard library imports
import asyncio
from copy import copy
from functools import partial
from operator import itemgetter

# 2. External package imports
import adaptive
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np

# 3. Internal imports
import funcs
import common
import adaptive_tools
adaptive.notebook_extension()

# Wire with intrinsic SC

In [ ]:
syst_pars = dict(a=10, L=3000, r=35, shape='circle', with_leads=True, L_barrier=0)

params = dict(alpha=70, B_x=0.5, B_y=0, B_z=0, Delta=0.25, g=50, 
              orbital=True, mu_sc=100, mu=10, mu_lead=10, c_tunnel=3/4,
              V=lambda *_: 0,
              V_barrier=common.gaussian, sigma=500, V_0=25, x0=0, **funcs.constants.__dict__)

syst = funcs.make_simple_3d_wire(**syst_pars)

kwant.plot(syst);

# Wire with external SC

In [ ]:
syst_pars = dict(a=10, angle=0, onsite_disorder=False, L_barrier=0,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

params = dict(alpha=70, B_x=0.5, B_y=0, B_z=0, Delta=60, g=50,
              orbital=True, mu_sc=100, mu=10, mu_lead=10, c_tunnel=3/4,
              V=lambda *_: 0,
              V_barrier=common.gaussian, sigma=500, V_0=25, x0=0, **funcs.constants.__dict__)


syst = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst);

# Pfaffian: topological phase diagram

In [ ]:
from functools import partial

lead_pars = dict(a=10, angle=0, coverage_angle=135, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True,
                 rotate_spin_orbit=False)

params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=10, mu_lead=10, mu_sc=100, c_tunnel=3/4, V=lambda *x: 0,
              intrinsic_sc=True,
              **funcs.constants.__dict__)



def fix_params(params, lead_pars):
    params = copy(params)
    lead_pars = copy(lead_pars)
    params['Delta'] = 0.25
    lead_pars['r'] = lead_pars['r1']
    for k in ['coverage_angle', 'angle', 'r1', 'r2', 'A_correction', 'with_shell']:
        lead_pars.pop(k, None)
    lead_pars['superconducting'] = True
    return params, lead_pars


def get_pfaffians(x, keys, val, params=params, lead_pars=lead_pars):
    import common
    import funcs
    import collections
    import scipy.sparse.linalg as sla

    params = common.parse_params({**params, **val})

    if not isinstance(x, collections.Iterable):
        x = [x]
    for k, v in zip(keys, x):
        params[k] = v

    if params['intrinsic_sc']:
        make_lead = funcs.make_simple_lead
        params, lead_pars = fix_params(params, lead_pars)
    else:
        make_lead = funcs.make_lead

    lead = make_lead(**lead_pars).finalized()

    return funcs.calculate_pfaffian(lead, params)


vals = common.named_product(orbital=[True, False],
                            intrinsic_sc=[True, False])

learners = []
for val in vals:
    f = partial(get_pfaffians, val=val, keys=['B_x', 'mu_lead'])
    learner = adaptive_tools.Learner2D(f, ([0, 4], [7, 40]))
    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)
folder = 'data-quasi/pfaffian'
learner.load(folder)

# For use in `Smallest gap phase diagram`
contours = {tuple(val.values()): hv.Path(funcs.get_contours_from_image(l.plot().Image.I))
            for val, l in zip(vals, learners)}
contours_keys = tuple(vals[0].keys())

In [ ]:
runner = adaptive.Runner(learner)
runner.live_info(update_interval=5)

In [ ]:
runner.task.print_stack(), [(l.npoints, l.loss()) for l in learners]

In [ ]:
%%output size=120

def plotter(learner):
    p = learner.plot().Image.I
    contours = hv.Path(funcs.get_contours_from_image(p))
    return p * hv.Path(contours)

HLines = hv.Overlay([hv.HLine(y) for y in [9.7, 14, 15.35]])
(learner.plot(cdims=vals, plotter=plotter) * HLines).layout().cols(2)

In [ ]:
learner.save(folder)

# Wavefunctions in the lead

In [ ]:
lead = syst.leads[0]
funcs.plot_wfs_in_cross_section(lead, params, 0)

# Finite sized system spectrum

In [ ]:
params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50,
              orbital=True, mu=None, mu_lead=None, mu_sc=100, c_tunnel=3/4,
              V='lambda *_: 0',
              V_barrier=common.gaussian, sigma=500, V_0=25, x0=0,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False, L_barrier=0,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)


def get_energies(x, key, val, params, syst_pars):
    import common
    import funcs
    import scipy.sparse.linalg as sla
    params = common.parse_params({**params, **val})

    params[key] = x

    if params['intrinsic_sc']:
        params, syst_pars = funcs.fix_simple_params(params, syst_pars)
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)

    ham = syst.hamiltonian_submatrix(params=params, sparse=True).tocsc()
    energies, _ = sla.eigsh(ham, sigma=0, k=30)
    return energies


vals = common.named_product(orbital=[True, False],
                            intrinsic_sc=[True, False],
                            mu=[9.7, 14, 15.35])

learners = []
for val in vals:
    f = partial(get_energies, key='B_x', val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner1D(f, [0, 4], adaptive.learner.learner1D.uniform_loss)
    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)
folder = 'data-quasi/finite-size-spectrum'
learner.load(folder)

# Potential shape

In [ ]:
V = lambda x: common.gaussian(x, params['V_0'], syst_pars['L_barrier'], params['sigma'])

hv.Curve([(x, V(x)) for x in range(syst_pars['L'] + syst_pars['L_barrier'])], vdims=['V'])

In [ ]:
runner = adaptive.Runner(learner, executor=client)

In [ ]:
runner.live_info(update_interval=5)

In [ ]:
runner.task.print_stack()

In [ ]:
[(l.npoints, l.loss()) for l in learners]

In [ ]:
# %%output size=120

def band_plotter(learner):
    Bs = list(learner.data.keys())
    es = np.array([e for e in learner.data.values()])
    plots = [hv.Scatter((Bs, e), kdims=['B_x'], vdims=['E']
                       ).opts(style=dict(color='k')) for e in es.T]
    return hv.Overlay(plots)

learner.plot(cdims=vals, plotter=band_plotter).select(E=(-.5, .5)
    ).layout(['orbital', 'intrinsic_sc']).cols(2)

In [ ]:
learner.save(folder)

In [ ]:
runner.task.print_stack(), learner.loss()

# Smallest gap phase diagram


In [ ]:
def smallest_gap(x, keys, val, params, syst_pars):
    import common
    import funcs
    import numpy as np
    import scipy.sparse.linalg as sla

    params = common.parse_params({**params, **val})

    for _x, key in zip(x, keys):
        params[key] = _x

    if params['intrinsic_sc']:
        params, syst_pars = funcs.fix_simple_params(params, syst_pars)
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)

    ham = syst.hamiltonian_submatrix(params=params, sparse=True).tocsc()
    evals, _ = sla.eigsh(ham, sigma=0, k=30, which='LM')
    return np.abs(evals).min()

In [ ]:
params = dict(alpha=70, B_y=0, B_z=0, Delta=60, g=50,
              mu_lead=14, mu_sc=100, c_tunnel=3/4,
              V='lambda *_: 0',
              V_barrier=common.gaussian, V_0=25, x0=0,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False, L_barrier=0,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = common.named_product(intrinsic_sc=[True, False],
                            orbital=[True, False],
                            sigma=[200, 300, 400, 500, 600])

learners = []
for val in vals:
    f = partial(smallest_gap, keys=['B_x', 'mu'],
                val=val,
                params=copy(params),
                syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, ([0, 4], [7, 20]))
    learner.cdims = val

    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)
folder = 'data-quasi/sigma_sweep'
learner.load(folder)

In [ ]:
runner = adaptive.Runner(learner, executor=client)

In [ ]:
runner.live_info(update_interval=5)

In [ ]:
runner.task.print_stack()

In [ ]:
learner.save(folder)

__run cell with Pfaffian before running the cell below for the phase boundaries!__

In [ ]:
%%opts Image [colorbar=True]
def phase_plotter(l, zlims=(0.01, 1)):
    im = l.plot(tri_alpha=0).Image.I
    im.data = np.log10(np.clip(im.data, *zlims))
    return (im.redim(x='B_x', y='mu') 
            * contours[tuple(l.cdims[k] for k in contours_keys)])[:, 7:20]

learner.plot(cdims=vals, plotter=phase_plotter).layout(['intrinsic_sc', 'orbital']).cols(2)

# Conductance

In [ ]:
params['sigma'] = 200
params['V_0'] = 4
syst_pars['L_barrier'] = 3 * params['sigma']

V = partial(common.gaussian, a=params['V_0'], mu=syst_pars['L_barrier'], sigma=params['sigma'])

# V = lambda x: (params['V_0'] * np.tanh((x - syst_pars['L_barrier']) / params['sigma']))
hv.Curve([(x, V(x)) for x in range(syst_pars['L'] + syst_pars['L_barrier'])], vdims=['V'])

In [ ]:
%%px --local
def conductance(x, val, syst_pars, params):
    import funcs

    assert syst_pars['L_barrier'] != 0, "Need a finite length barrier!"

    for k in ['angle']:
        try:
            syst_pars[k] = val[k]
        except KeyError:
            pass

    params = funcs.parse_params(dict(**params, **val))
    
    params['mu_lead'] = params['mu'] - params['V'](0)
    params['B_x'], params['B_y'], params['B_z'] = (0, 0, 0)
    val['V_bias'], params['B_{}'.format(params['direction'])] = x

    if params['intrinsic_sc']:
        params, syst_pars = funcs.fix_simple_params(params, syst_pars)
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)
    return funcs.andreev_conductance(syst, params, E=val['V_bias'])

def conductance1d(x, val, syst_pars, params):
    x, y = x, val['B_x']
    return conductance(x, val, syst_pars, params)

In [ ]:
params = dict(alpha=70, B_y=0, B_z=0, Delta=60, g=50,
              mu_sc=100, c_tunnel=3/4,
              V='lambda *_: 0', direction='x',
              V_barrier=common.gaussian, sigma=200, x0=0,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False, L_barrier=3*params['sigma'],
                 L=4000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True,
                 right_lead=False)

In [ ]:
# Learner2D

vals = common.named_product(orbital=[True, False],
                            intrinsic_sc=[True, False],
                            V_0=[0, 1, 2, 3, 4, 5],
                            mu=[9.7, 14, 15.35])

learners = []
for val in vals:
    bounds = [[-1, 1], [0, 4]]
    f = partial(conductance,
                val=copy(val),
                params=copy(params),
                syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, bounds)
    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)

folder = 'data-quasi/conductance-2D'
learner.load(folder)

In [ ]:
# Learner1D

vals = common.named_product(orbital=[True],
                            intrinsic_sc=[True],
                            V_0=np.linspace(0, 5, 20),
                            mu=np.arange(8, 20),
                            B_x=[0, 0.5, 1, 1.5, 2, 2.5, 3])

learners = []
for val in vals:
    f = partial(conductance,
                val=copy(val),
                params=copy(params),
                syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner1D(f, (-1, 1))
    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)
folder = 'data-quasi/conductance-1D'
learner.load(folder)

In [ ]:
len(learners)

In [ ]:
%%opts EdgePaths (color='w')
learner.plot(cdims=vals)

## Run 

In [ ]:
runner = adaptive.Runner(learner, executor=client)
runner.live_info(update_interval=5)

In [ ]:
runner.task.print_stack()

In [ ]:
folder = 'data-quasi/conductance-2D'
learner.save(folder)

In [ ]:
%%opts Image [colorbar=True]
plotter = lambda l: l.plot().redim(x='E', y='B_x')
learner.plot(cdims=vals, plotter=plotter).layout(['intrinsic_sc', 'orbital']).cols(2)

##  Run different runners

In [ ]:
import distributed
cluster = distributed.LocalCluster(n_workers=30)
ex = distributed.Client(cluster)

In [ ]:
futs = adaptive_tools.split_learners_in_executor(learners, ex, profile='pbs', ncores=30, save_interval=900)

In [ ]:
x = [task.done() for task in futs if task.done()]
x

In [ ]:
adaptive_tools.combine_learners_from_folders(
    learners, file_pattern='tmp-*/*',
    save_folder=folder,
    save_fname_pattern='data_learner_{}.pickle')

In [ ]:
learner.save('data-quasi/conductance')

In [ ]:
runner.task.print_stack()

In [ ]:
%%opts Image [colorbar=True]
%%output size=200

def plotter(learner):
    return learner.plot(tri_alpha=0.).Image.I.redim(x='V_bias', y='B_x', z='G')

learner.plot(plotter)

# Band structure

In [ ]:
# %%opts Path [aspect='square']
params = dict(c_tunnel=3/4, B_x=0.5, B_y=0, B_z=0, V_barrier=40, g=50, mu_sc=100,
              alpha=70, orbital=False, V=lambda x, y, z: 10 * z / 35, Delta=60,
              **funcs.constants.__dict__)

lead_pars = dict(a=10, angle=0,
                 coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_shell=True, A_correction=True)

lead = funcs.make_lead(**lead_pars).finalized()
params['mu_lead'] = params['mu_sc'] = 15
params['B_y'] = 0
ks = np.linspace(-1, 1)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

p1[:, -25:25]# + p1[:, -2:2]

# Tuning the superconducting gap

In [ ]:
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=70, orbital=True, V='lambda x, y, z: 10 * z / 35', mu_sc=100,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, coverage_angle=135, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True)

def lowest_energy(x, syst_pars, params):
    import funcs
    import numpy as np
    lead = funcs.make_lead(**syst_pars).finalized()
    params['mu_lead'], params['Delta'] = x
    # Combine the fixed parameters `params` and the changing
    # parameters `val` to one dict and evaluate the string
    # lambda functions.
    params = funcs.parse_params(params)

    # Create the Hamiltonian `ham` at k=0.
    h0 = lead.cell_hamiltonian(params=params)
    t0 = lead.inter_cell_hopping(params=params)
    ham = h0 + t0 + t0.conj().T

    # Find the energies.
    ev = np.linalg.eigvalsh(ham)
    
    # Return a combined dictionary with the results and input.
    return np.abs(ev).min()

def find_crossings(f, g):
    return np.argwhere(np.diff(np.sign(f - g)) != 0).reshape(-1)

def get_Delta(learner, Delta_ind):
    plot = learner.plot(n=1000).redim(x='mu', y='Delta', z='E_gap')
    line = plot.reduce(['mu'], np.min)
    idx = find_crossings(line.data['E_gap'], Delta_ind)[-1]
    Delta = line.data['Delta'][idx]
    return Delta

In [ ]:
learner = adaptive_tools.Learner2D(partial(lowest_energy, syst_pars=syst_pars, params=params),
                             [(5, 20), (0, 200)])

runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.001)

In [ ]:
learner.function((0,0))

In [ ]:
runner.task.print_stack()

In [ ]:
%%opts Image [aspect='square']
plot = learner.plot().redim(x='mu', y='Delta', z='E_gap')
plot

In [ ]:
%%opts Scatter (size=10, color='r')
Delta_ind = 0.4

line = plot.reduce(['mu'], np.min)
line * hv.HLine(0.4) * hv.Scatter((get_Delta(learner, Delta_ind), Delta_ind))

In [ ]:
Deltas = np.arange(*learner.bounds[1])
hm = hv.HoloMap({Delta: plot.sample(Delta=Delta)[:, 0:1] for Delta in Deltas})
hm * hv.HLine(0.4)